### load gpt2 model

In [1]:
import torch
import torch.nn as nn
from GPT2 import GPT2Model, GPT2Tokenizer
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = ''
device = 'cuda' #'cuda'


def tokenize_input(inputStr, tokenizer, seq_length=1024):
    pad_id = tokenizer.encoder['<pad>']
    tokenized_sentence = tokenizer.encode(inputStr)[:seq_length-20]
    tokens = tokenized_sentence
    token_length = len(tokens)
    tokens.extend([pad_id] * (seq_length - token_length))
    tokens = torch.tensor(tokens, dtype=torch.long)
    return tokens.reshape(1,1024), [token_length]

tokenizer = GPT2Tokenizer(
    'GPT2/bpe/vocab.json',
    'GPT2/bpe/chinese_vocab.model',
    max_len=512)
    
model = GPT2Model(
    vocab_size=30000,
    layer_size=12,
    block_size=1024,
    embedding_dropout=0.0,
    embedding_size=768,
    num_attention_heads=12,
    attention_dropout=0.0,
    residual_dropout=0.0
)

state_dict = torch.load('../models/model_pretrain_distill.pth', map_location='cpu')
model.load_state_dict(state_dict)

model.eval()

model.to(device)

print('loaded success')

loaded success


### demo

In [ ]:
# output the vector

inputStr = '这股票估计会大跌'  # the text you want to classify

tokens, token_length = tokenize_input(inputStr, tokenizer, seq_length=1024)
output = model(tokens.to(device))

vector = output[0,token_length[0]]
vector.detach().cpu().numpy()

### get vectors for all documents

In [2]:
import numpy as np
from tqdm import tqdm

data = np.load('./data/eastmoney_full_stocks_list_nlu_tencent.pkl', allow_pickle=True)

In [3]:
data[0]['baike_summary']

'浙江康德莱医疗器械股份有限公司成立于1987年，是康德莱集团属下一家专业生产一次性针类系列医疗器械的制造厂家。主要是生产加工,经销批发医疗器械。'

In [4]:
all = []

for idx in tqdm(range(len(data))):
    try:
        content = data[idx]['baike_content'] + data[idx]['baike_summary']
        tokens, token_length = tokenize_input(content, tokenizer, seq_length=1024)
        output = model(tokens.to(device))
        vector = output[0,0:token_length[0]].sum(0).detach().cpu().numpy()
        one = {
            'ticker_id': data[idx]['ticker_id'],
            'ticker_name': data[idx]['ticker_name'],
            'vector': vector
        }
        all.append(one)
    except Exception as err:
        print(err)

  0%|          | 0/510 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.522 seconds.
Prefix dict has been built successfully.
100%|██████████| 510/510 [01:04<00:00,  7.87it/s]


In [6]:
import pickle

with open('./data_ignore/vectors_gpt2_sum.pkl','wb') as f:
    pickle.dump(all, f)

### query by vector matrix

In [7]:
import numpy as np
from tqdm import tqdm

data = np.load('./data_ignore/vectors_gpt2_sum.pkl', allow_pickle=True)

In [8]:
data[0]

{'ticker_id': '603987',
 'ticker_name': '康德莱',
 'vector': array([ 1927.2593, -4370.8687, -4326.062 , ..., -3482.5044, -3108.6743,
        -5288.259 ], dtype=float32)}

In [9]:
ticker_names = []
ticker_ids = []
vectors = []

for idx in tqdm(range(len(data))):
    if 'vector' in data[idx].keys():
        vectors.append(data[idx]['vector'])
        ticker_ids.append(data[idx]['ticker_id'])
        ticker_names.append(data[idx]['ticker_name'])


100%|██████████| 510/510 [00:00<00:00, 329395.60it/s]


In [10]:
vectors = np.stack(vectors)

In [11]:
def similarity_vector_matrix(arr, brr):
    return arr.dot(brr.T) / (np.sqrt(np.sum(arr*arr)) * np.sqrt(np.sum(brr*brr, axis=1)))

In [12]:
def stock_search(query, topk=10):

    tokens, token_length = tokenize_input(query, tokenizer, seq_length=1024)
    output = model(tokens.to(device))
    vector = output[0,token_length[0]].detach().cpu().numpy()

    res = similarity_vector_matrix(vector, vectors)
    idxs = np.argsort(res)[::-1]

    topk_idxs = idxs[:topk]
    names = [ticker_names[idx] for idx in topk_idxs]
    print(names)

- 自动驾驶，新能源汽车
- 电影，电视剧，文化艺术
- 啤酒，烧烤，朋友聚会
- 医疗保险，重大疾病保障
- 新冠肺炎

In [13]:
query = '自动驾驶，新能源汽车'
stock_search(query)

['森麒麟', '长青股份', '瑞芯微', '广和通', '鼎阳科技', '东华科技', '传音控股', '长城汽车', '海亮股份', '京东方A']


In [14]:
query = '电影，电视剧，文化艺术'
stock_search(query)

['中国化学', '长青股份', '火星人', '蓝色光标', '传音控股', '完美世界', '金山办公', '来伊份', '海亮股份', 'TCL科技']


In [15]:
query = '啤酒，烧烤，朋友聚会'
stock_search(query)

['火星人', '香飘飘', '重庆啤酒', '同花顺', '长青股份', '来伊份', '完美世界', '李子园', '森马服饰', '妙可蓝多']


In [16]:
query = '医疗保险，重大疾病保障'
stock_search(query)

['长青股份', '中国化学', '海亮股份', '巨化股份', 'TCL科技', '水晶光电', '东睦股份', '四川双马', '天坛生物', '金山办公']


In [17]:
query = '新冠肺炎'
stock_search(query)

['长青股份', '中国化学', '可立克', '康希诺', '一品红', '司太立', '火星人', '海亮股份', '蓝色光标', '森麒麟']


### 实验结果分析
直接加在一起作为篇章向量效果很差，没有原来一个的效果好

### TODO
- 搜索篇章向量的方法
- 搜索怎么做相似文档检索
- doc2vec
- [参考](https://zhuanlan.zhihu.com/p/80737146) jieba除去停用词

1、向量检索这部分本质上就是一个anns的问题，查找距离最近的向量再排序。

2、Es7里面这部分的支持本身是一个实验性质的，这个在官网上游说明的，而且这部分也是是个暴利计算的knn的过程，向量维度过大，或者量在几万到几十万的时候时间就开始有点长了，量稍微一大基本没法用的。

3、回到问题得本质anns这类问题的本质处理，还是要做优质的索引结构来减低检索的复杂度和时间，lsh和kdtree、pq这种主要是对精度损失有点大，目前趋势还是在图索引上的发展，比较有代表还是hnsw以及阿里和浙大的一起出的nsg和nsg的算法，hnsw这个就是内存有点大，但是有开源比较好的实现。

4、至于es的插件实现，有暴利的knn的插件，大概率大家可能都是用的这种，因为es7以下的只能这么弄，写脚本的，这个速度肯定起不来的，再就是在脚本中实现索引算法和实现的，相对麻烦，因为要从插入，检索，索引增量等多个角度来从插件来扩展es，这种事比较完美的。

5、还有就是这部分不管是暴利的还是算法实现，底层用c的代码实现（avx等等指令集）要比java快至少在几十倍往上，

6、再就是一些算法的并发检索等一些在量上和检索速度上的优化。